<a href="https://colab.research.google.com/github/krishna-gera/my-aiml-learning/blob/main/day-29/day29_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import joblib

saved = joblib.load("day28_models.joblib")
print("Keys inside joblib file:", saved.keys())



Keys inside joblib file: dict_keys(['preprocessor', 'stacking_model', 'best_threshold', 'selected_rule'])


In [9]:
import gradio as gr
import pandas as pd
import joblib
import traceback

# Load saved models
try:
    saved = joblib.load("day28_models.joblib")
    preprocessor = saved.get("preprocessor")
    final_model = saved.get("stacking_model")
    best_threshold = saved.get("best_threshold", 0.5)  # default 0.5 if missing

    if final_model is None:
        raise ValueError("No final model found in joblib file.")
    if preprocessor is None:
        raise ValueError("Preprocessor not found in joblib file.")

except Exception:
    print("Error loading models:")
    print(traceback.format_exc())
    final_model = None
    preprocessor = None
    best_threshold = 0.5

expected_cols = list(preprocessor.feature_names_in_) if preprocessor else []

def predict_file(file):
    try:
        if final_model is None or preprocessor is None:
            return "⚠️ Models or preprocessor not loaded properly."

        df = pd.read_csv(file.name)
        df = df.loc[:, ~df.columns.str.contains("^Unnamed")]

        # Ensure all expected columns exist
        for col in expected_cols:
            if col not in df.columns:
                df[col] = 0
        df = df[expected_cols]

        # Transform + predict probabilities
        X_proc = preprocessor.transform(df)
        if hasattr(final_model, "predict_proba"):
            probs = final_model.predict_proba(X_proc)[:, 1]
            preds = (probs >= best_threshold).astype(int)
        else:
            preds = final_model.predict(X_proc)

        df["Survived"] = preds
        output_path = "day29_predictions.csv"
        df.to_csv(output_path, index=False)
        return output_path

    except Exception:
        return f"⚠️ Error:\n{traceback.format_exc()}"

demo = gr.Interface(
    fn=predict_file,
    inputs=gr.File(file_types=[".csv"], label="Upload CSV"),
    outputs="file",
    title="🚢 Titanic Survival Prediction (Day 29)",
    description="Upload CSV and download predictions with Survived column."
)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://29377c5846f66ae85e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
